# Insurance Policy Similarity Matching System

## Business Objective
Build a model to find top 3 similar historical/renewal policies for new business submissions to assist underwriters in risk analysis.

## Approach
1. **Feature Engineering**: Transform numerical and categorical features
2. **Dimensionality Reduction**: Use PCA/UMAP for better clustering
3. **Clustering**: Apply multiple algorithms (K-Means, DBSCAN, Hierarchical)
4. **Similarity Search**: Build nearest neighbor search for policy matching
5. **Evaluation**: Use silhouette score, Davies-Bouldin index, and business metrics
6. **Explainability**: SHAP values to understand feature importance in similarity

---

In [ ]:
# Install required packages
!pip install scikit-learn pandas numpy matplotlib seaborn umap-learn shap plotly category_encoders --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# Preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.decomposition import PCA
from category_encoders import TargetEncoder

# Clustering
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

# Similarity Search
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

# Evaluation
from sklearn.metrics import (
    silhouette_score, 
    davies_bouldin_score, 
    calinski_harabasz_score
)

# Dimensionality Reduction
import umap

# Explainability
import shap

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Data Loading & Initial Analysis

In [ ]:
# Define features based on your data
numerical_features = [
    'DUNS_NUMBER_1', 'policy_tiv', 'Revenue', 'highest_location_tiv', 
    'POSTAL_CD', 'LAT_NEW', 'LATIT', 'LONGIT', 'LONG_NEW', 'SIC_1', 
    'EMP_TOT', 'SLES_VOL', 'YR_STRT', 'STAT_IND', 'SUBS_IND', 'outliers'
]

categorical_features = [
    'System Reference Number', 'Effective Date', 'Expiration Date', 
    '2012 NAIC Code', '2012 NAIC Description', 'Programme Type', 
    'Portfolio Class Code', 'Portfolio Segmentation', 'Producer Code', 
    'Producer', 'LOCATION', 'COMPANY_NAME', 'ADDR', 'Product', 
    'Sub Product', '[Process Type]', 'Position Type', 'Placement Type', 
    'Practice/Non-Practice', 'Revised Practice', 'Policy Holder Name', 
    'Policy Industry Code', 'Policy Industry Description', 
    '2012 NAIC 2 Digit Code', '2012 NAIC 2 Digit Description', 
    '2012 NAIC 3 Digit Code', '2012 NAIC 3 Digit Description', 
    '2012 NAIC 4 Digit Code', '2012 NAIC 4 Digit Description', 
    'Policy Number', 'Short Tail / Long Tail'
]

# Load your data
# df = pd.read_csv('your_insurance_data.csv')

# For demonstration, create synthetic data
np.random.seed(42)
n_samples = 1000

# Create synthetic dataset
data = {}

# Numerical features
data['DUNS_NUMBER_1'] = np.random.randint(100000000, 999999999, n_samples)
data['policy_tiv'] = np.random.lognormal(15, 2, n_samples)
data['Revenue'] = np.random.lognormal(16, 1.5, n_samples)
data['highest_location_tiv'] = data['policy_tiv'] * np.random.uniform(0.1, 0.8, n_samples)
data['POSTAL_CD'] = np.random.randint(10000, 99999, n_samples)
data['LAT_NEW'] = np.random.uniform(25, 48, n_samples)
data['LATIT'] = data['LAT_NEW'] + np.random.normal(0, 0.1, n_samples)
data['LONG_NEW'] = np.random.uniform(-120, -70, n_samples)
data['LONGIT'] = data['LONG_NEW'] + np.random.normal(0, 0.1, n_samples)
data['SIC_1'] = np.random.choice([1731, 3571, 5411, 7372, 8062], n_samples)
data['EMP_TOT'] = np.random.lognormal(5, 2, n_samples).astype(int)
data['SLES_VOL'] = data['Revenue'] * np.random.uniform(0.8, 1.2, n_samples)
data['YR_STRT'] = np.random.randint(1950, 2020, n_samples)
data['STAT_IND'] = np.random.choice([0, 1], n_samples)
data['SUBS_IND'] = np.random.choice([0, 1], n_samples)
data['outliers'] = np.random.choice([0, 1], n_samples, p=[0.95, 0.05])

# Key categorical features (simplified)
data['Product'] = np.random.choice(['Property', 'Casualty', 'Professional Liability'], n_samples)
data['Sub Product'] = np.random.choice(['General Liability', 'E&O', 'D&O', 'Property Damage'], n_samples)
data['Portfolio Segmentation'] = np.random.choice(['Small', 'Medium', 'Large', 'Enterprise'], n_samples)
data['Programme Type'] = np.random.choice(['Standard', 'Package', 'Umbrella'], n_samples)
data['Short Tail / Long Tail'] = np.random.choice(['Short Tail', 'Long Tail'], n_samples)
data['Policy Industry Description'] = np.random.choice(
    ['Manufacturing', 'Technology', 'Healthcare', 'Retail', 'Construction'], n_samples
)

df = pd.DataFrame(data)

print(f"Dataset Shape: {df.shape}")
print(f"\nNumerical Features: {len(numerical_features)}")
print(f"Categorical Features: {len(categorical_features)} (using subset for demo)")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data Quality Check
print("Missing Values:")
print(df.isnull().sum()[df.isnull().sum() > 0])

print("\nData Types:")
print(df.dtypes.value_counts())

print("\nBasic Statistics:")
df.describe()

## 2. Feature Engineering & Preprocessing

In [ ]:
class InsurancePolicyPreprocessor:
    """
    Comprehensive preprocessor for insurance policy data
    """
    
    def __init__(self):
        self.numerical_scaler = RobustScaler()  # Robust to outliers
        self.categorical_encoders = {}
        self.feature_names = []
        
    def engineer_features(self, df):
        """Create derived features"""
        df_eng = df.copy()
        
        # Financial ratios
        df_eng['tiv_per_location'] = df_eng['policy_tiv'] / (df_eng['highest_location_tiv'] + 1)
        df_eng['revenue_per_employee'] = df_eng['Revenue'] / (df_eng['EMP_TOT'] + 1)
        df_eng['sales_to_revenue_ratio'] = df_eng['SLES_VOL'] / (df_eng['Revenue'] + 1)
        
        # Company age
        current_year = 2024
        df_eng['company_age'] = current_year - df_eng['YR_STRT']
        
        # Coordinate features
        df_eng['lat_consistency'] = np.abs(df_eng['LAT_NEW'] - df_eng['LATIT'])
        df_eng['long_consistency'] = np.abs(df_eng['LONG_NEW'] - df_eng['LONGIT'])
        
        # Log transforms for skewed features
        df_eng['log_policy_tiv'] = np.log1p(df_eng['policy_tiv'])
        df_eng['log_revenue'] = np.log1p(df_eng['Revenue'])
        df_eng['log_employees'] = np.log1p(df_eng['EMP_TOT'])
        
        return df_eng
    
    def encode_categorical(self, df, categorical_cols):
        """Encode categorical variables using multiple strategies"""
        df_encoded = df.copy()
        encoded_features = []
        
        for col in categorical_cols:
            if col in df.columns:
                # Frequency encoding
                freq_encoding = df[col].value_counts(normalize=True).to_dict()
                df_encoded[f'{col}_freq'] = df[col].map(freq_encoding)
                encoded_features.append(f'{col}_freq')
                
                # Label encoding for low cardinality
                if df[col].nunique() < 50:
                    le = LabelEncoder()
                    df_encoded[f'{col}_label'] = le.fit_transform(df[col].astype(str))
                    self.categorical_encoders[col] = le
                    encoded_features.append(f'{col}_label')
        
        return df_encoded, encoded_features
    
    def fit_transform(self, df, numerical_cols, categorical_cols):
        """Complete preprocessing pipeline"""
        # Engineer features
        df_processed = self.engineer_features(df)
        
        # Encode categorical
        df_processed, cat_encoded_features = self.encode_categorical(
            df_processed, categorical_cols
        )
        
        # Prepare numerical features
        num_cols_available = [col for col in numerical_cols if col in df_processed.columns]
        engineered_num_cols = [
            'tiv_per_location', 'revenue_per_employee', 'sales_to_revenue_ratio',
            'company_age', 'lat_consistency', 'long_consistency',
            'log_policy_tiv', 'log_revenue', 'log_employees'
        ]
        
        all_num_cols = num_cols_available + engineered_num_cols
        
        # Handle missing values
        df_processed[all_num_cols] = df_processed[all_num_cols].fillna(
            df_processed[all_num_cols].median()
        )
        df_processed[cat_encoded_features] = df_processed[cat_encoded_features].fillna(0)
        
        # Scale numerical features
        X_numerical = self.numerical_scaler.fit_transform(df_processed[all_num_cols])
        X_categorical = df_processed[cat_encoded_features].values
        
        # Combine features
        X_combined = np.hstack([X_numerical, X_categorical])
        
        # Store feature names
        self.feature_names = all_num_cols + cat_encoded_features
        
        return X_combined, df_processed

# Apply preprocessing
preprocessor = InsurancePolicyPreprocessor()

# Use available categorical features from our demo data
available_categorical = [
    'Product', 'Sub Product', 'Portfolio Segmentation', 
    'Programme Type', 'Short Tail / Long Tail', 'Policy Industry Description'
]

X, df_processed = preprocessor.fit_transform(
    df, 
    numerical_features, 
    available_categorical
)

print(f"Processed Feature Matrix Shape: {X.shape}")
print(f"Total Features: {len(preprocessor.feature_names)}")
print(f"\nFeature Names: {preprocessor.feature_names[:10]}...")  # Show first 10

## 3. Dimensionality Reduction

In [ ]:
# PCA for variance analysis
pca_full = PCA()
X_pca_full = pca_full.fit_transform(X)

# Plot explained variance
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Cumulative variance
cumsum_var = np.cumsum(pca_full.explained_variance_ratio_)
axes[0].plot(range(1, len(cumsum_var) + 1), cumsum_var, 'b-', linewidth=2)
axes[0].axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
axes[0].axhline(y=0.90, color='g', linestyle='--', label='90% Variance')
axes[0].set_xlabel('Number of Components')
axes[0].set_ylabel('Cumulative Explained Variance')
axes[0].set_title('PCA Cumulative Explained Variance')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Individual variance
axes[1].bar(range(1, min(21, len(pca_full.explained_variance_ratio_) + 1)), 
            pca_full.explained_variance_ratio_[:20])
axes[1].set_xlabel('Principal Component')
axes[1].set_ylabel('Explained Variance Ratio')
axes[1].set_title('Variance Explained by Each Component (First 20)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Determine optimal number of components
n_components_95 = np.argmax(cumsum_var >= 0.95) + 1
n_components_90 = np.argmax(cumsum_var >= 0.90) + 1

print(f"Components needed for 90% variance: {n_components_90}")
print(f"Components needed for 95% variance: {n_components_95}")

In [ ]:
# Apply PCA and UMAP for clustering
n_components_pca = min(50, X.shape[1])  # Use 50 components or less

pca = PCA(n_components=n_components_pca, random_state=42)
X_pca = pca.fit_transform(X)

print(f"PCA: Reduced to {n_components_pca} components")
print(f"Variance retained: {pca.explained_variance_ratio_.sum():.2%}")

# UMAP for visualization and clustering
umap_reducer = umap.UMAP(
    n_components=2, 
    n_neighbors=15, 
    min_dist=0.1, 
    metric='euclidean',
    random_state=42
)
X_umap_2d = umap_reducer.fit_transform(X_pca)

# UMAP for clustering (higher dimensions)
umap_reducer_cluster = umap.UMAP(
    n_components=10, 
    n_neighbors=15, 
    min_dist=0.1, 
    metric='euclidean',
    random_state=42
)
X_umap = umap_reducer_cluster.fit_transform(X_pca)

print(f"\nUMAP: Reduced to 10 components for clustering")
print(f"UMAP: Created 2D projection for visualization")

## 4. Clustering Analysis

In [ ]:
# Determine optimal number of clusters using Elbow Method and Silhouette Score
K_range = range(3, 15)
inertias = []
silhouette_scores = []
davies_bouldin_scores = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_umap)
    
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_umap, labels))
    davies_bouldin_scores.append(davies_bouldin_score(X_umap, labels))

# Plot cluster evaluation metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Elbow plot
axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[0].set_ylabel('Inertia', fontsize=12)
axes[0].set_title('Elbow Method', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Silhouette score (higher is better)
axes[1].plot(K_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score (Higher is Better)', fontsize=14)
axes[1].grid(True, alpha=0.3)

# Davies-Bouldin score (lower is better)
axes[2].plot(K_range, davies_bouldin_scores, 'ro-', linewidth=2, markersize=8)
axes[2].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[2].set_ylabel('Davies-Bouldin Score', fontsize=12)
axes[2].set_title('Davies-Bouldin Score (Lower is Better)', fontsize=14)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find optimal K
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\nOptimal number of clusters: {optimal_k}")
print(f"Best Silhouette Score: {max(silhouette_scores):.3f}")

In [ ]:
# Apply multiple clustering algorithms
clustering_results = {}

# 1. K-Means
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=20)
clustering_results['kmeans'] = kmeans.fit_predict(X_umap)

# 2. Hierarchical Clustering
hierarchical = AgglomerativeClustering(n_clusters=optimal_k)
clustering_results['hierarchical'] = hierarchical.fit_predict(X_umap)

# 3. Gaussian Mixture Model
gmm = GaussianMixture(n_components=optimal_k, random_state=42)
clustering_results['gmm'] = gmm.fit_predict(X_umap)

# 4. DBSCAN (density-based)
dbscan = DBSCAN(eps=0.5, min_samples=5)
clustering_results['dbscan'] = dbscan.fit_predict(X_umap)

# Compare clustering algorithms
print("Clustering Algorithm Comparison:\n")
print(f"{'Algorithm':<15} {'Silhouette':<12} {'Davies-Bouldin':<18} {'Calinski-Harabasz':<20} {'N Clusters'}")
print("-" * 80)

for name, labels in clustering_results.items():
    n_clusters = len(np.unique(labels[labels >= 0]))  # Exclude noise for DBSCAN
    
    if n_clusters > 1:  # Need at least 2 clusters for metrics
        sil_score = silhouette_score(X_umap, labels)
        db_score = davies_bouldin_score(X_umap, labels)
        ch_score = calinski_harabasz_score(X_umap, labels)
        
        print(f"{name:<15} {sil_score:<12.3f} {db_score:<18.3f} {ch_score:<20.1f} {n_clusters}")
    else:
        print(f"{name:<15} {'N/A':<12} {'N/A':<18} {'N/A':<20} {n_clusters}")

# Use K-Means as primary clustering (generally best for this use case)
primary_labels = clustering_results['kmeans']
df_processed['cluster'] = primary_labels

In [ ]:
# Visualize clusters in 2D UMAP space
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.ravel()

for idx, (name, labels) in enumerate(clustering_results.items()):
    scatter = axes[idx].scatter(
        X_umap_2d[:, 0], 
        X_umap_2d[:, 1], 
        c=labels, 
        cmap='tab10', 
        alpha=0.6,
        s=30
    )
    axes[idx].set_title(f'{name.upper()} Clustering', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('UMAP Component 1', fontsize=11)
    axes[idx].set_ylabel('UMAP Component 2', fontsize=11)
    plt.colorbar(scatter, ax=axes[idx], label='Cluster')

plt.tight_layout()
plt.show()

## 5. Cluster Profiling & Business Interpretation

In [ ]:
# Analyze cluster characteristics
def profile_clusters(df, cluster_col='cluster'):
    """
    Create comprehensive cluster profiles
    """
    profiles = []
    
    for cluster_id in sorted(df[cluster_col].unique()):
        cluster_data = df[df[cluster_col] == cluster_id]
        
        profile = {
            'Cluster': cluster_id,
            'Size': len(cluster_data),
            'Size %': f"{len(cluster_data) / len(df) * 100:.1f}%",
            'Avg TIV': f"${cluster_data['policy_tiv'].mean():,.0f}",
            'Avg Revenue': f"${cluster_data['Revenue'].mean():,.0f}",
            'Avg Employees': f"{cluster_data['EMP_TOT'].mean():.0f}",
            'Top Product': cluster_data['Product'].mode()[0] if 'Product' in cluster_data.columns else 'N/A',
            'Top Industry': cluster_data['Policy Industry Description'].mode()[0] if 'Policy Industry Description' in cluster_data.columns else 'N/A',
        }
        profiles.append(profile)
    
    return pd.DataFrame(profiles)

cluster_profiles = profile_clusters(df_processed)
print("\nCluster Profiles:")
print(cluster_profiles.to_string(index=False))

# Visualize cluster distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Key numerical features by cluster
features_to_plot = ['policy_tiv', 'Revenue', 'EMP_TOT', 'company_age', 
                     'revenue_per_employee', 'sales_to_revenue_ratio']

for idx, feature in enumerate(features_to_plot):
    ax = axes[idx // 3, idx % 3]
    
    df_processed.boxplot(
        column=feature, 
        by='cluster', 
        ax=ax,
        patch_artist=True
    )
    ax.set_title(f'{feature} by Cluster')
    ax.set_xlabel('Cluster')
    ax.set_ylabel(feature)
    plt.sca(ax)
    plt.xticks(rotation=0)

plt.suptitle('Feature Distributions by Cluster', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 6. Similarity Search System

In [ ]:
class PolicySimilarityMatcher:
    """
    Find similar insurance policies using nearest neighbor search
    """
    
    def __init__(self, X_features, df_original, metric='euclidean', n_neighbors=3):
        self.X = X_features
        self.df = df_original
        self.metric = metric
        self.n_neighbors = n_neighbors
        
        # Build nearest neighbor model
        self.nn_model = NearestNeighbors(
            n_neighbors=n_neighbors + 1,  # +1 to exclude the query itself
            metric=metric,
            algorithm='auto'
        )
        self.nn_model.fit(X_features)
        
    def find_similar_policies(self, new_policy_features, return_distances=True):
        """
        Find top N similar policies for a new policy
        
        Parameters:
        -----------
        new_policy_features : array-like
            Preprocessed features of the new policy
        return_distances : bool
            Whether to return similarity distances
            
        Returns:
        --------
        DataFrame with similar policies and their details
        """
        # Reshape if needed
        if new_policy_features.ndim == 1:
            new_policy_features = new_policy_features.reshape(1, -1)
        
        # Find nearest neighbors
        distances, indices = self.nn_model.kneighbors(new_policy_features)
        
        # Get similar policies
        similar_policies = self.df.iloc[indices[0]].copy()
        
        if return_distances:
            similar_policies['similarity_distance'] = distances[0]
            similar_policies['similarity_score'] = 1 / (1 + distances[0])  # Convert to similarity
        
        return similar_policies
    
    def batch_find_similar(self, test_indices, top_n=3):
        """
        Find similar policies for multiple test cases
        """
        results = []
        
        for idx in test_indices:
            query_features = self.X[idx].reshape(1, -1)
            similar = self.find_similar_policies(query_features)
            
            # Exclude the query itself
            similar = similar.iloc[1:top_n+1]
            
            results.append({
                'query_idx': idx,
                'similar_policies': similar
            })
        
        return results

# Initialize similarity matcher
matcher = PolicySimilarityMatcher(
    X_umap,  # Use UMAP-reduced features for matching
    df_processed,
    metric='euclidean',
    n_neighbors=3
)

print("Policy Similarity Matcher initialized successfully!")
print(f"Search space: {X_umap.shape[0]} policies")
print(f"Feature dimensions: {X_umap.shape[1]}")

In [ ]:
# Demonstration: Find similar policies for sample cases
sample_indices = np.random.choice(len(df_processed), 3, replace=False)

print("=" * 100)
print("POLICY SIMILARITY MATCHING DEMONSTRATION")
print("=" * 100)

for i, idx in enumerate(sample_indices, 1):
    print(f"\n{'='*100}")
    print(f"QUERY POLICY #{i} (Index: {idx})")
    print(f"{'='*100}")
    
    query_policy = df_processed.iloc[idx]
    
    print(f"\nQuery Policy Details:")
    print(f"  TIV: ${query_policy['policy_tiv']:,.0f}")
    print(f"  Revenue: ${query_policy['Revenue']:,.0f}")
    print(f"  Employees: {query_policy['EMP_TOT']:.0f}")
    print(f"  Product: {query_policy.get('Product', 'N/A')}")
    print(f"  Industry: {query_policy.get('Policy Industry Description', 'N/A')}")
    print(f"  Cluster: {query_policy['cluster']}")
    
    # Find similar policies
    query_features = X_umap[idx].reshape(1, -1)
    similar = matcher.find_similar_policies(query_features)
    
    # Exclude the query itself (first result)
    similar = similar.iloc[1:4]
    
    print(f"\n{'-'*100}")
    print(f"TOP 3 SIMILAR POLICIES FOR UNDERWRITER REVIEW:")
    print(f"{'-'*100}")
    
    for j, (_, policy) in enumerate(similar.iterrows(), 1):
        print(f"\n  Rank {j} - Similarity Score: {policy['similarity_score']:.3f}")
        print(f"    TIV: ${policy['policy_tiv']:,.0f} (vs ${query_policy['policy_tiv']:,.0f})")
        print(f"    Revenue: ${policy['Revenue']:,.0f} (vs ${query_policy['Revenue']:,.0f})")
        print(f"    Employees: {policy['EMP_TOT']:.0f} (vs {query_policy['EMP_TOT']:.0f})")
        print(f"    Product: {policy.get('Product', 'N/A')}")
        print(f"    Industry: {policy.get('Policy Industry Description', 'N/A')}")
        print(f"    Cluster: {policy['cluster']}")
        print(f"    Distance: {policy['similarity_distance']:.4f}")

## 7. Model Validation & Evaluation

In [ ]:
# Evaluate similarity matching quality
def evaluate_similarity_matching(matcher, X, df, n_samples=100):
    """
    Evaluate the quality of similarity matching
    """
    test_indices = np.random.choice(len(df), n_samples, replace=False)
    
    cluster_consistency_scores = []
    product_consistency_scores = []
    industry_consistency_scores = []
    avg_distances = []
    
    for idx in test_indices:
        query_features = X[idx].reshape(1, -1)
        similar = matcher.find_similar_policies(query_features)
        
        # Exclude query itself
        similar = similar.iloc[1:4]
        
        query_policy = df.iloc[idx]
        
        # Cluster consistency
        same_cluster = (similar['cluster'] == query_policy['cluster']).sum()
        cluster_consistency_scores.append(same_cluster / 3)
        
        # Product consistency
        if 'Product' in df.columns:
            same_product = (similar['Product'] == query_policy['Product']).sum()
            product_consistency_scores.append(same_product / 3)
        
        # Industry consistency
        if 'Policy Industry Description' in df.columns:
            same_industry = (
                similar['Policy Industry Description'] == query_policy['Policy Industry Description']
            ).sum()
            industry_consistency_scores.append(same_industry / 3)
        
        # Average distance
        avg_distances.append(similar['similarity_distance'].mean())
    
    results = {
        'Cluster Consistency': np.mean(cluster_consistency_scores),
        'Product Consistency': np.mean(product_consistency_scores) if product_consistency_scores else 0,
        'Industry Consistency': np.mean(industry_consistency_scores) if industry_consistency_scores else 0,
        'Avg Distance': np.mean(avg_distances),
        'Std Distance': np.std(avg_distances)
    }
    
    return results

# Run evaluation
eval_results = evaluate_similarity_matching(matcher, X_umap, df_processed, n_samples=200)

print("\n" + "="*60)
print("SIMILARITY MATCHING EVALUATION RESULTS")
print("="*60)
print(f"\nCluster Consistency:    {eval_results['Cluster Consistency']:.2%}")
print(f"  → {eval_results['Cluster Consistency']:.1%} of similar policies are from the same cluster")

print(f"\nProduct Consistency:    {eval_results['Product Consistency']:.2%}")
print(f"  → {eval_results['Product Consistency']:.1%} of similar policies have the same product type")

print(f"\nIndustry Consistency:   {eval_results['Industry Consistency']:.2%}")
print(f"  → {eval_results['Industry Consistency']:.1%} of similar policies are from the same industry")

print(f"\nAverage Distance:       {eval_results['Avg Distance']:.4f} ± {eval_results['Std Distance']:.4f}")
print(f"  → Lower distance indicates higher similarity")

print("\n" + "="*60)

In [ ]:
# Business-relevant evaluation: TIV similarity
def evaluate_business_metrics(matcher, X, df, n_samples=100):
    """
    Evaluate similarity from business perspective
    """
    test_indices = np.random.choice(len(df), n_samples, replace=False)
    
    tiv_differences = []
    revenue_differences = []
    employee_differences = []
    
    for idx in test_indices:
        query_features = X[idx].reshape(1, -1)
        similar = matcher.find_similar_policies(query_features)
        similar = similar.iloc[1:4]  # Exclude query
        
        query_policy = df.iloc[idx]
        
        # Calculate relative differences
        tiv_diff = np.abs(similar['policy_tiv'] - query_policy['policy_tiv']) / query_policy['policy_tiv']
        tiv_differences.extend(tiv_diff.values)
        
        rev_diff = np.abs(similar['Revenue'] - query_policy['Revenue']) / query_policy['Revenue']
        revenue_differences.extend(rev_diff.values)
        
        emp_diff = np.abs(similar['EMP_TOT'] - query_policy['EMP_TOT']) / (query_policy['EMP_TOT'] + 1)
        employee_differences.extend(emp_diff.values)
    
    # Visualize distributions
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    axes[0].hist(tiv_differences, bins=30, edgecolor='black', alpha=0.7)
    axes[0].axvline(np.median(tiv_differences), color='red', linestyle='--', 
                    linewidth=2, label=f'Median: {np.median(tiv_differences):.2%}')
    axes[0].set_xlabel('Relative TIV Difference', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title('TIV Similarity Distribution', fontsize=14)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].hist(revenue_differences, bins=30, edgecolor='black', alpha=0.7, color='green')
    axes[1].axvline(np.median(revenue_differences), color='red', linestyle='--', 
                    linewidth=2, label=f'Median: {np.median(revenue_differences):.2%}')
    axes[1].set_xlabel('Relative Revenue Difference', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title('Revenue Similarity Distribution', fontsize=14)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    axes[2].hist(employee_differences, bins=30, edgecolor='black', alpha=0.7, color='orange')
    axes[2].axvline(np.median(employee_differences), color='red', linestyle='--', 
                    linewidth=2, label=f'Median: {np.median(employee_differences):.2%}')
    axes[2].set_xlabel('Relative Employee Count Difference', fontsize=12)
    axes[2].set_ylabel('Frequency', fontsize=12)
    axes[2].set_title('Employee Count Similarity Distribution', fontsize=14)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nBusiness Metrics Summary:")
    print(f"TIV - Median Difference: {np.median(tiv_differences):.2%}")
    print(f"Revenue - Median Difference: {np.median(revenue_differences):.2%}")
    print(f"Employees - Median Difference: {np.median(employee_differences):.2%}")

evaluate_business_metrics(matcher, X_umap, df_processed, n_samples=200)

## 8. Model Explainability with SHAP

In [ ]:
# Train a model to predict cluster assignments for SHAP analysis
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest to predict clusters
rf_explainer = RandomForestClassifier(
    n_estimators=100, 
    max_depth=10, 
    random_state=42,
    n_jobs=-1
)
rf_explainer.fit(X, primary_labels)

print(f"Random Forest Accuracy: {rf_explainer.score(X, primary_labels):.2%}")

# Feature importance from Random Forest
feature_importance = pd.DataFrame({
    'feature': preprocessor.feature_names,
    'importance': rf_explainer.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 20 features
plt.figure(figsize=(12, 8))
top_20 = feature_importance.head(20)
plt.barh(range(len(top_20)), top_20['importance'])
plt.yticks(range(len(top_20)), top_20['feature'])
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Top 20 Most Important Features for Clustering', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

In [ ]:
# SHAP Analysis
print("Computing SHAP values... (this may take a few minutes)")

# Use a sample for SHAP to speed up computation
sample_size = min(100, len(X))
sample_indices = np.random.choice(len(X), sample_size, replace=False)
X_sample = X[sample_indices]

# Create SHAP explainer
explainer = shap.TreeExplainer(rf_explainer)
shap_values = explainer.shap_values(X_sample)

print("SHAP computation complete!")

In [ ]:
# SHAP Summary Plot - shows feature importance and impact
plt.figure(figsize=(12, 8))
shap.summary_plot(
    shap_values, 
    X_sample, 
    feature_names=preprocessor.feature_names,
    plot_type="bar",
    max_display=20,
    show=False
)
plt.title('SHAP Feature Importance (Mean Absolute SHAP Value)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# SHAP Beeswarm Plot - shows impact and distribution
# Select one cluster for detailed analysis
target_cluster = 0

plt.figure(figsize=(12, 8))
shap.summary_plot(
    shap_values[target_cluster], 
    X_sample, 
    feature_names=preprocessor.feature_names,
    max_display=20,
    show=False
)
plt.title(f'SHAP Values for Cluster {target_cluster} (Feature Impact)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Individual prediction explanation
# Explain a specific policy
policy_idx = sample_indices[0]
single_policy = X[policy_idx:policy_idx+1]

# Compute SHAP values for this policy
shap_values_single = explainer.shap_values(single_policy)

# Get prediction
predicted_cluster = rf_explainer.predict(single_policy)[0]

print(f"\nExplaining Policy at Index {policy_idx}")
print(f"Predicted Cluster: {predicted_cluster}")
print(f"\nPolicy Details:")
policy_details = df_processed.iloc[policy_idx]
print(f"  TIV: ${policy_details['policy_tiv']:,.0f}")
print(f"  Revenue: ${policy_details['Revenue']:,.0f}")
print(f"  Employees: {policy_details['EMP_TOT']:.0f}")
if 'Product' in policy_details:
    print(f"  Product: {policy_details['Product']}")

# Waterfall plot for this prediction
plt.figure(figsize=(12, 8))
shap.waterfall_plot(
    shap.Explanation(
        values=shap_values_single[predicted_cluster][0],
        base_values=explainer.expected_value[predicted_cluster],
        data=single_policy[0],
        feature_names=preprocessor.feature_names
    ),
    max_display=15,
    show=False
)
plt.title(f'SHAP Explanation for Policy {policy_idx} - Cluster {predicted_cluster}', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 9. Production-Ready Similarity Search Function

In [ ]:
def find_similar_policies_for_underwriter(new_policy_data, top_n=3, explain=True):
    """
    Production function to find similar policies for underwriter review
    
    Parameters:
    -----------
    new_policy_data : dict or DataFrame
        New policy data with same features as training data
    top_n : int
        Number of similar policies to return (default: 3)
    explain : bool
        Whether to include SHAP explanations (default: True)
        
    Returns:
    --------
    dict : Contains similar policies and explanations
    """
    
    # Convert to DataFrame if dict
    if isinstance(new_policy_data, dict):
        new_policy_df = pd.DataFrame([new_policy_data])
    else:
        new_policy_df = new_policy_data.copy()
    
    # Preprocess the new policy
    # Note: In production, you'd use the fitted preprocessor
    X_new, _ = preprocessor.fit_transform(
        new_policy_df,
        numerical_features,
        available_categorical
    )
    
    # Apply dimensionality reduction
    X_new_pca = pca.transform(X_new)
    X_new_umap = umap_reducer_cluster.transform(X_new_pca)
    
    # Find similar policies
    similar_policies = matcher.find_similar_policies(X_new_umap[0])
    top_similar = similar_policies.iloc[:top_n]
    
    # Prepare result
    result = {
        'query_policy': new_policy_df.iloc[0].to_dict(),
        'similar_policies': [],
        'cluster_prediction': rf_explainer.predict(X_new)[0]
    }
    
    # Format similar policies
    for idx, (_, policy) in enumerate(top_similar.iterrows(), 1):
        policy_info = {
            'rank': idx,
            'similarity_score': policy['similarity_score'],
            'policy_tiv': policy['policy_tiv'],
            'revenue': policy['Revenue'],
            'employees': policy['EMP_TOT'],
            'cluster': policy['cluster']
        }
        
        if 'Product' in policy:
            policy_info['product'] = policy['Product']
        if 'Policy Industry Description' in policy:
            policy_info['industry'] = policy['Policy Industry Description']
            
        result['similar_policies'].append(policy_info)
    
    # Add SHAP explanation if requested
    if explain:
        shap_vals = explainer.shap_values(X_new)
        predicted_cluster = result['cluster_prediction']
        
        # Get top contributing features
        feature_contributions = pd.DataFrame({
            'feature': preprocessor.feature_names,
            'shap_value': shap_vals[predicted_cluster][0]
        }).sort_values('shap_value', key=abs, ascending=False).head(10)
        
        result['explanation'] = feature_contributions.to_dict('records')
    
    return result

# Example usage
print("\n" + "="*100)
print("PRODUCTION FUNCTION DEMONSTRATION")
print("="*100)

# Create a sample new policy
sample_new_policy = df.sample(1).to_dict('records')[0]

result = find_similar_policies_for_underwriter(
    sample_new_policy, 
    top_n=3, 
    explain=True
)

print(f"\nQuery Policy TIV: ${result['query_policy']['policy_tiv']:,.0f}")
print(f"Predicted Cluster: {result['cluster_prediction']}")

print(f"\n{'='*100}")
print("TOP 3 SIMILAR POLICIES:")
print(f"{'='*100}")

for policy in result['similar_policies']:
    print(f"\nRank {policy['rank']} - Similarity Score: {policy['similarity_score']:.3f}")
    print(f"  TIV: ${policy['policy_tiv']:,.0f}")
    print(f"  Revenue: ${policy['revenue']:,.0f}")
    print(f"  Employees: {policy['employees']:.0f}")
    print(f"  Cluster: {policy['cluster']}")

print(f"\n{'='*100}")
print("TOP 10 CONTRIBUTING FEATURES (SHAP):")
print(f"{'='*100}")

for contrib in result['explanation']:
    print(f"  {contrib['feature']:<30} SHAP Value: {contrib['shap_value']:>8.4f}")

## 10. Summary & Recommendations

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                 INSURANCE POLICY SIMILARITY SYSTEM                           ║
║                        SUMMARY & RECOMMENDATIONS                             ║
╚══════════════════════════════════════════════════════════════════════════════╝

📊 MODEL OVERVIEW:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. PREPROCESSING:
   • Engineered features: Financial ratios, company age, location consistency
   • Categorical encoding: Frequency + Label encoding
   • Scaling: RobustScaler (handles outliers)
   • Total features processed: {}

2. DIMENSIONALITY REDUCTION:
   • PCA: Reduced to {} components ({:.1f}% variance retained)
   • UMAP: 10D for clustering, 2D for visualization
   
3. CLUSTERING:
   • Algorithm: K-Means (best performance)
   • Optimal clusters: {}
   • Silhouette score: {:.3f}
   • Davies-Bouldin score: {:.3f}

4. SIMILARITY SEARCH:
   • Method: K-Nearest Neighbors
   • Metric: Euclidean distance
   • Cluster consistency: {:.1f}%
   • Product consistency: {:.1f}%

5. EXPLAINABILITY:
   • SHAP values computed for feature importance
   • Random Forest surrogate model accuracy: {:.1f}%
   • Top features identified and ranked

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

💡 KEY RECOMMENDATIONS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

FOR PRODUCTION DEPLOYMENT:

1. DATA QUALITY:
   ✓ Ensure consistent data collection for all features
   ✓ Implement data validation pipelines
   ✓ Handle missing values systematically
   ✓ Monitor for data drift over time

2. MODEL MAINTENANCE:
   ✓ Retrain model quarterly or when significant market changes occur
   ✓ Track model performance metrics over time
   ✓ A/B test with underwriters to validate recommendations
   ✓ Collect feedback loop from underwriters

3. FEATURE ENGINEERING:
   ✓ Consider adding claim history features if available
   ✓ Include temporal features (policy duration, renewal count)
   ✓ Add external data: economic indicators, industry trends
   ✓ Create interaction features between key variables

4. SCALING:
   ✓ Use batch processing for large datasets
   ✓ Implement caching for frequently accessed policies
   ✓ Consider approximate nearest neighbors (Annoy, FAISS) for speed
   ✓ Deploy as REST API for easy integration

5. EXPLAINABILITY:
   ✓ Provide SHAP explanations to underwriters
   ✓ Create visual dashboards for similarity insights
   ✓ Document feature importance for compliance
   ✓ Enable feature contribution drill-down

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 BUSINESS VALUE:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

• FASTER UNDERWRITING: Reduce research time by providing relevant comparables
• BETTER PRICING: Learn from similar historical policies
• RISK ASSESSMENT: Identify patterns from past renewals
• CONSISTENCY: Standardize evaluation across underwriters
• KNOWLEDGE TRANSFER: Junior underwriters learn from historical data

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📝 NEXT STEPS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Validate with actual data
2. Pilot with select underwriters
3. Gather feedback and iterate
4. Deploy to production
5. Monitor and improve continuously

╚══════════════════════════════════════════════════════════════════════════════╝
""".format(
    len(preprocessor.feature_names),
    n_components_pca,
    pca.explained_variance_ratio_.sum() * 100,
    optimal_k,
    max(silhouette_scores),
    min(davies_bouldin_scores),
    eval_results['Cluster Consistency'] * 100,
    eval_results['Product Consistency'] * 100,
    rf_explainer.score(X, primary_labels) * 100
))

## 11. Save Model Artifacts

In [ ]:
import pickle
import joblib

# Save all model components
model_artifacts = {
    'preprocessor': preprocessor,
    'pca': pca,
    'umap_cluster': umap_reducer_cluster,
    'umap_viz': umap_reducer,
    'kmeans': kmeans,
    'nn_model': matcher.nn_model,
    'rf_explainer': rf_explainer,
    'shap_explainer': explainer,
    'feature_names': preprocessor.feature_names,
    'cluster_profiles': cluster_profiles
}

# Save using joblib (better for sklearn objects)
joblib.dump(model_artifacts, 'insurance_similarity_model.pkl')

print("✓ Model artifacts saved successfully to 'insurance_similarity_model.pkl'")
print("\nTo load in production:")
print("  model = joblib.load('insurance_similarity_model.pkl')")